In [ ]:
"""
# list houses - DistHouse
PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX c:  <http://iec.ch/TC57/CIM100#>
SELECT ?fdrname ?name ?parent ?coolingSetpoint ?coolingSystem ?floorArea ?heatingSetpoint ?heatingSystem ?hvacPowerFactor ?numberOfStories ?thermalIntegrity ?id ?fdrid 
WHERE { 
#VALUES ?fdrid {"_E407CBB6-8C8D-9BC9-589C-AB83FBF0826D"}  # IEEE123 PV/Triplex
#VALUES ?fdrid {"_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3"}  # 8500-node
#VALUES ?fdrid {"_AAE94E4A-2465-6F5E-37B1-3E72183A4E44"}  # 9500-node
#VALUES ?fdrid {"_9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095"}  # R2
#VALUES ?fdrid {"_503D6E20-F499-4CC7-8051-971E23D0BF79"}  # 123 Transactive
   ?h r:type c:House.
   ?h c:IdentifiedObject.name ?name.
   ?h c:IdentifiedObject.mRID ?id.
   ?h c:House.floorArea ?floorArea.
   ?h c:House.numberOfStories ?numberOfStories.
   OPTIONAL{?h c:House.coolingSetpoint ?coolingSetpoint.}
   OPTIONAL{?h c:House.heatingSetpoint ?heatingSetpoint.}
   OPTIONAL{?h c:House.hvacPowerFactor ?hvacPowerFactor.}
   ?h c:House.coolingSystem ?coolingSystemRaw.
   	bind(strafter(str(?coolingSystemRaw),"HouseCooling.") as ?coolingSystem) 
   ?h c:House.heatingSystem ?heatingSystemRaw.
   	bind(strafter(str(?heatingSystemRaw),"HouseHeating.") as ?heatingSystem)
   ?h c:House.thermalIntegrity ?thermalIntegrityRaw.
   	bind(strafter(str(?thermalIntegrityRaw),"HouseThermalIntegrity.") as ?thermalIntegrity)
   ?h c:House.EnergyConsumer ?econ.
   ?econ c:IdentifiedObject.name ?parent.
   ?fdr c:IdentifiedObject.mRID ?fdrid.
   ?fdr c:IdentifiedObject.name ?fdrname.
   ?econ c:Equipment.EquipmentContainer ?fdr.
} ORDER BY ?fdrname ?name
"""

In [ ]:
"""
# loads (need to account for 2+ unequal EnergyConsumerPhases per EnergyConsumer) - DistLoad
PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX c:  <http://iec.ch/TC57/CIM100#>
SELECT ?name ?bus ?basev ?p ?q ?conn ?cnt ?pz ?qz ?pi ?qi ?pp ?qp ?pe ?qe ?fdrid (group_concat(distinct ?phs;separator="\n") as ?phases) WHERE {
 ?s r:type c:EnergyConsumer.
# feeder selection options - if all commented out, query matches all feeders
#VALUES ?fdrid {"_9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095"}  # R2 12.47 3
 ?s c:Equipment.EquipmentContainer ?fdr.
 ?fdr c:IdentifiedObject.mRID ?fdrid.
 ?s c:IdentifiedObject.name ?name.
 ?s c:ConductingEquipment.BaseVoltage ?bv.
 ?bv c:BaseVoltage.nominalVoltage ?basev.
 ?s c:EnergyConsumer.customerCount ?cnt.
 ?s c:EnergyConsumer.p ?p.
 ?s c:EnergyConsumer.q ?q.
 ?s c:EnergyConsumer.phaseConnection ?connraw.
   bind(strafter(str(?connraw),"PhaseShuntConnectionKind.") as ?conn)
 ?s c:EnergyConsumer.LoadResponse ?lr.
 ?lr c:LoadResponseCharacteristic.pConstantImpedance ?pz.
 ?lr c:LoadResponseCharacteristic.qConstantImpedance ?qz.
 ?lr c:LoadResponseCharacteristic.pConstantCurrent ?pi.
 ?lr c:LoadResponseCharacteristic.qConstantCurrent ?qi.
 ?lr c:LoadResponseCharacteristic.pConstantPower ?pp.
 ?lr c:LoadResponseCharacteristic.qConstantPower ?qp.
 ?lr c:LoadResponseCharacteristic.pVoltageExponent ?pe.
 ?lr c:LoadResponseCharacteristic.qVoltageExponent ?qe.
 OPTIONAL {?ecp c:EnergyConsumerPhase.EnergyConsumer ?s.
 ?ecp c:EnergyConsumerPhase.phase ?phsraw.
   bind(strafter(str(?phsraw),"SinglePhaseKind.") as ?phs) }
 ?t c:Terminal.ConductingEquipment ?s.
 ?t c:Terminal.ConnectivityNode ?cn. 
 ?cn c:IdentifiedObject.name ?bus
}
GROUP BY ?name ?bus ?basev ?p ?q ?cnt ?conn ?pz ?qz ?pi ?qi ?pp ?qp ?pe ?qe ?fdrid
ORDER by ?name
"""